In [1]:
from dotenv import load_dotenv
import dspy
import mlflow

from agentic_system.agents import (
    CFEfficacyAgent,
    ToxicityScreeningAgent,
    CompoundPrioritizationAgent,
)

In [2]:
# NOTE: Start MLflow server with:
# mlflow server --backend-store-uri sqlite:///mydb.sqlite
# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Create a unique name for your experiment.
mlflow.set_experiment("Optimize Efficacy Tools")
mlflow.autolog()
mlflow.tracing.disable_notebook_display()

load_dotenv("../.env")
lm = dspy.LM("gemini/gemini-2.5-pro", temperature=0.9, cache=False)
dspy.settings.configure(lm=lm, track_usage=True)

2025/10/06 17:24:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/10/06 17:24:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/10/06 17:24:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/10/06 17:24:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.
2025/10/06 17:24:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [3]:
COMPOUND = "Veliparib"

efficacy_agent = CFEfficacyAgent()
efficacy_results = efficacy_agent(compound_name=COMPOUND)
print(efficacy_results)

/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/_internal/_known_annotated_metadata.py:106: RuntimeWarning: coroutine '_fetch_pubmed_data.<locals>._async_fetch' was never awaited
  def expand_grouped_metadata(annotations: Iterable[Any]) -> Iterable[Any]:
/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/_internal/_repr.py:63: RuntimeWarning: coroutine '_fetch_pubmed_data.<locals>._async_fetch' was never awaited
  return join_str.join(repr(v) if a is None else f'{a}={v!r}' for a, v in self.__repr_args__())
/Users/roshankern/Desktop/Github/cf-compound-selection-demo/backend/.venv/lib/python3.12/site-packages/pydantic/_internal/_repr.py:63: RuntimeWarning: coroutine '_fetch_pubmed_data.<locals>._async_fetch' was never awaited
  return join_str.join(repr(v) if a is None else f'{a}={v!r}' for a, v in self.__repr_args__())


Prediction(
    trajectory={'thought_0': 'The user wants me to assess the efficacy of Veliparib for reversing the failing cardiac fibroblast phenotype.\n\nMy first step is to consult the internal LITL database. This is the highest priority source of information as it contains data from the exact same assay. I will use `LITL__rag_query` to search for any information related to Veliparib or its mechanism of action in the context of cardiac fibrosis.', 'tool_name_0': 'LITL__rag_query', 'tool_args_0': {'query': 'What is the efficacy of PARP inhibitors like Veliparib in reversing cardiac fibroblast activation or cardiac fibrosis?', 'compound_to_exclude': 'Veliparib'}, 'observation_0': 'Run 1:\n# Agent Run Summary\n## Compound\nVemurafenib\n\n## Trajectory\n- Step 0:\n    - I need to determine the efficacy of Vemurafenib in reversing the failing cardiac fibroblast phenotype. My first step is to understand the compound\'s mechanism of action. To do this, I will first find its ChEMBL ID, which

In [4]:
steps = len(efficacy_results.trajectory) // 4
cost = sum([x["cost"] for x in lm.history])

print("Steps:", steps)
print("Total Cost (USD):", cost)

Steps: 10
Total Cost (USD): 0.3069690625


In [5]:
# tox_agent = ToxicityScreeningAgent(max_iters=12)
# tox_results = tox_agent(compound_name=COMPOUND)
# print(tox_results)

In [6]:
# prioritization_agent = CompoundPrioritizationAgent()
# prioritization_results = prioritization_agent(compound_name=COMPOUND)
# print(prioritization_results)